This notebook is being used for studying purposes. I am following medium post
[Predicting Invasive Ductal Carcinoma using Convolutional Neural Network (CNN) in Keras](https://towardsdatascience.com/predicting-invasive-ductal-carcinoma-using-convolutional-neural-network-cnn-in-keras-debb429de9a6) and the medium post's [github repository](https://github.com/bikramb98/invasive-ductal-carcinoma-cnn/blob/master/Predicting%20Invasive%20Ductal%20Carcinoma%20using%20CNN%20in%20Keras.ipynb) by Bikram Baruah. The dataset used can be found [here](http://www.andrewjanowczyk.com/use-case-6-invasive-ductal-carcinoma-idc-segmentation/).

### Loading the dataset:

---



In [1]:
from glob import glob
import fnmatch
import cv2

<span style='color:blue'>
    <b>Study Notes:</b></span>
    
glob module: [doc]( https://docs.python.org/3/library/glob.html)
and [source](https://github.com/python/cpython/blob/3.8/Lib/glob.py)<br>
fnmatch module: [doc](https://docs.python.org/3/library/fnmatch.html#fnmatch.fnmatch) and [source](https://github.com/python/cpython/blob/3.8/Lib/fnmatch.py)


In [2]:
# saves pathnames
image_patches = glob('./idc_regular_ps50_idx5/IDC_regular_ps50_idx5/*/*/*.png')

pattern_zero = '*class0.png'
pattern_one = '*class1.png'

<span style='color:blue'>
    <b>Study Notes:</b></span>
    
The function glob is retrieving the files' pathnames according to a specific pattern. The variables pattern_[blank] are string variables that, as the name indicates, are going to be used to find files of a certain pattern. In this case, ending with either 'class0.png' or 'class1.png'.

In [3]:
print(f'Lenght: {len(image_patches)} and exemple of path: {image_patches[1]}')

Lenght: 277524 and exemple of path: ./idc_regular_ps50_idx5/IDC_regular_ps50_idx5/12823/0/12823_idx5_x1501_y2351_class0.png


In [4]:
# Saves the image file location of all images according to its class (0 or 1)
class_zero = fnmatch.filter(image_patches, pattern_zero)
class_one = fnmatch.filter(image_patches, pattern_one)

<span style='color:blue'>
    <b>Study Notes:</b></span>

Here we're filtering the list of pathnames by its class.


In [5]:
def process_images(lower_index, upper_index):
    height, width, channels = 50, 50, 3
    
    x = [] #list to store image data
    y = [] #list to store labels
    
    for img in image_patches[lower_index:upper_index]:
        full_size_image = cv2.imread(img)
        image = (cv2.resize(full_size_image, (width, height), interpolation = cv2.INTER_CUBIC))
        
        x.append(image)
    
        if img in class_zero:
            y.append(0)
        elif img in class_one:
            y.append(1)
        else:
            return

    return x, y

<span style='color:blue'>
    <b>Study Notes:</b></span>
    
Will have to look more into cv2 and OpenCV later. From what I've gathered, imread loads an image, as the name somewhat indicates. Now INTER_CUBIC seems to be a type of interpolation, I have found a little about it [here](https://chadrick-kwag.net/cv2-resize-interpolation-methods/), but will have to read and research it a little bit better. 

<span style='color:red'>Must come back later to this note.</span>

In [6]:
X, Y = process_images(0, 60000)

### Data preprocessing
***

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

<span style='color:blue'>
    <b>Study Notes:</b></span>
    
More about train_test_split [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). But it's very self explanatory, it splits the arrays into random train and test subsets. Using the parameter test_size you can indicate the proportion of the dateset that will be included in the test subset. There is a train_size as well that works the same way, except for the train subset. There are other interesting parameters to try and test it out later. 

<span style='color:red'>Must come back later to this note.</span>



In [8]:
X = np.array(X)
X = X.astype(np.float32)
X /= 255. # Ensures values between 0 and 1

In [19]:
# Splits training and test set, reserves 15% of the dataset for testing
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.15) 

In [10]:
print(f'There are {y_train.count(1)} 1s and {y_train.count(0)} 0s.\nLenght: {len(y_train)}')

There are 15057 1s and 35943 0s.
Lenght: 51000


In [11]:
import keras
from keras.utils import to_categorical

<span style='color:blue'>
    <b>Study Notes:</b></span>

Here we are one-hot-enconding the output. For future reference, read [this](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/).

In [21]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [13]:
print(f'x_train shape:{x_train.shape}\nx_test shape:{x_test.shape}')

x_train shape:(51000, 50, 50, 3)
x_test shape:(9000, 50, 50, 3)


In [15]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

In [16]:
x_test_flat.shape

(9000, 7500)

In [25]:
from imblearn.under_sampling import RandomUnderSampler

ModuleNotFoundError: No module named 'imblearn'

<span style='color:blue'>
    <b>Study session observations</b></span>
    
    
Search more about Google Colab and how to work with a big database. The kernel crashed a few times and it was a bit annoying. It happened almost always while using the train_test_split function. 

While not necessarily adjacent to the notebook, look into the possibility of using the notebook outside of the conda environment. Had a few problems downloading certain modules through `conda`. 